In [ ]:
!pip install transformers
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# Mount Google Drive
drive.mount('/content/drive')

# Load RoBERTa tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('roberta-base')
model = AutoModel.from_pretrained('roberta-base')

# Generate RoBERTa sentence embeddings
def generate_roberta_embeddings(sentences):
    sentence_vectors = []
    for sentence in sentences:
        inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
            # Use the [CLS] token's embedding
            cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
        sentence_vectors.append(cls_embedding)
    return np.array(sentence_vectors)

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Individual model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    # Expanded parameter grid
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40, 50],
        'min_samples_split': [2, 5, 10, 15, 20],
        'min_samples_leaf': [1, 2, 4, 6, 8]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def catboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'iterations': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.3], 'depth': [4, 6, 10]}
    model = tune_model(CatBoostClassifier(verbose=0), param_grid, X_train, y_train)
    print("CatBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'CatBoost': catboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }
    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input file path
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_CONFIDENCE.xlsx'

# Load the dataset
df = pd.read_excel(input_file)

# Generate RoBERTa embeddings for the "answer" column
X = generate_roberta_embeddings(df['answer'])

# Modify y to make sure the labels start from 0
y = df['confidence'].apply(lambda x: x + 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Run all models and print metrics
run_all_models(X_train, y_train, X_test, y_test)


Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'adam', 'hidden_layer_sizes': (50,), 'activation': 'relu'}
MLP Model Metrics:
Train Accuracy: 0.9328
Train Precision: 0.9331
Train Recall: 0.9328
Train F1 Score: 0.9317
Train Confusion Matrix:
[[ 29   2   1]
 [  0 110  22]
 [  0   7 305]]

Test Accuracy: 0.7463
Test Precision: 0.7481
Test Recall: 0.7463
Test F1 Score: 0.7448
Test Confusion Matrix:
[[  3   7   3]
 [  2  21  18]
 [  2  20 129]]

Running XGBoost model...
Best Params: {'subsample': 0.5, 'n_estimators': 150, 'learning_rate': 0.2}
XGBoost Model Metrics:
Train Accuracy: 0.9958
Train Precision: 0.9958
Train Recall: 0.9958
Train F1 Score: 0.9958
Train Confusion Matrix:
[[ 32   0   0]
 [  0 130   2]
 [  0   0 312]]

Test Accuracy: 0.7512
Test Precision: 0.7073
Test Recall: 0.7512
Test F1 Score: 0.7232
Test Confusion Matrix:
[[  1   9   3]
 [  2  13  26]
 [  1  10 140]]

Running AdaBoost model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 9 is smaller than n_iter=30. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'n_estimators': 50, 'learning_rate': 1.0}
AdaBoost Model Metrics:
Train Accuracy: 0.7857
Train Precision: 0.7914
Train Recall: 0.7857
Train F1 Score: 0.7866
Train Confusion Matrix:
[[ 20   9   3]
 [  2  87  43]
 [  0  45 267]]

Test Accuracy: 0.7024
Test Precision: 0.7112
Test Recall: 0.7024
Test F1 Score: 0.7001
Test Confusion Matrix:
[[  2   7   4]
 [  1  18  22]
 [  1  26 124]]

Running CatBoost model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

# Mount Google Drive
drive.mount('/content/drive')

# Load RoBERTa tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('roberta-base')
model = AutoModel.from_pretrained('roberta-base')

# Generate RoBERTa sentence embeddings
def generate_roberta_embeddings(sentences):
    sentence_vectors = []
    for sentence in sentences:
        inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
            # Use the [CLS] token's embedding
            cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
        sentence_vectors.append(cls_embedding)
    return np.array(sentence_vectors)

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }
    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input file path
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_CONFIDENCE.xlsx'

# Load the dataset
df = pd.read_excel(input_file)

# Generate RoBERTa embeddings for the "answer" column
X = generate_roberta_embeddings(df['answer'])

# Modify y to make sure the labels start from 0
y = df['confidence'].apply(lambda x: x + 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Run all models and print metrics
run_all_models(X_train, y_train, X_test, y_test)


Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Running Logistic Regression model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 10}
Logistic Regression Model Metrics:
Train Accuracy: 0.8887
Train Precision: 0.8912
Train Recall: 0.8887
Train F1 Score: 0.8837
Train Confusion Matrix:
[[ 19   5   8]
 [  0  99  33]
 [  1   6 305]]

Test Accuracy: 0.7561
Test Precision: 0.7314
Test Recall: 0.7561
Test F1 Score: 0.7406
Test Confusion Matrix:
[[  2   7   4]
 [  2  17  22]
 [  2  13 136]]

Running SVM model...
Best Params: {'kernel': 'linear', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.9139
Train Precision: 0.9158
Train Recall: 0.9139
Train F1 Score: 0.9114
Train Confusion Matrix:
[[ 29   1   2]
 [  0 101  31]
 [  1   6 305]]

Test Accuracy: 0.7415
Test Precision: 0.7311
Test Recall: 0.7415
Test F1 Score: 0.7355
Test Confusion Matrix:
[[  3   6   4]
 [  3  17  21]
 [  7  12 132]]



In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.decomposition import PCA

# Mount Google Drive
drive.mount('/content/drive')

# Load RoBERTa tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('roberta-base')
model = AutoModel.from_pretrained('roberta-base')

# Generate RoBERTa sentence embeddings
def generate_roberta_embeddings(sentences):
    sentence_vectors = []
    for sentence in sentences:
        inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
            # Use the [CLS] token's embedding
            cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
        sentence_vectors.append(cls_embedding)
    return np.array(sentence_vectors)

def apply_pca(X, variance_threshold=0.95):
    pca = PCA()
    X_reduced = pca.fit_transform(X)

    # Determine number of components to preserve desired variance
    explained_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(explained_variance >= variance_threshold) + 1
    print(f"Number of components selected: {n_components}")

    # Reapply PCA with the determined number of components
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)
    return X_reduced, pca

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Individual model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    # Expanded parameter grid
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40, 50],
        'min_samples_split': [2, 5, 10, 15, 20],
        'min_samples_leaf': [1, 2, 4, 6, 8]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def catboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'iterations': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.3], 'depth': [4, 6, 10]}
    model = tune_model(CatBoostClassifier(verbose=0), param_grid, X_train, y_train)
    print("CatBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }
    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input file path
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_CONFIDENCE.xlsx'

# Load the dataset
df = pd.read_excel(input_file)

# Generate RoBERTa embeddings for the "answer" column
X = generate_roberta_embeddings(df['answer'])

# Apply PCA for dimensionality reduction
X_pca, pca_model = apply_pca(X, variance_threshold=0.95)

# Modify y to make sure the labels start from 0
y = df['confidence'].apply(lambda x: x + 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)

# Run all models and print metrics
run_all_models(X_train, y_train, X_test, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Number of components selected: 135
Running KNN model...


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best Params: {'weights': 'uniform', 'n_neighbors': 3}
KNN Model Metrics:
Train Accuracy: 0.8235
Train Precision: 0.8281
Train Recall: 0.8235
Train F1 Score: 0.8180
Train Confusion Matrix:
[[ 23   5   4]
 [ 17  75  40]
 [  5  13 294]]

Test Accuracy: 0.6732
Test Precision: 0.6777
Test Recall: 0.6732
Test F1 Score: 0.6742
Test Confusion Matrix:
[[  2   8   3]
 [ 10   9  22]
 [  9  15 127]]

Running Random Forest model...
Best Params: {'n_estimators': 50, 'min_samples_split': 2, 'max_depth': 20}
Random Forest Model Metrics:
Train Accuracy: 0.9958
Train Precision: 0.9958
Train Recall: 0.9958
Train F1 Score: 0.9958
Train Confusion Matrix:
[[ 32   0   0]
 [  0 130   2]
 [  0   0 312]]

Test Accuracy: 0.7073
Test Precision: 0.5995
Test Recall: 0.7073
Test F1 Score: 0.6390
Test Confusion Matrix:
[[  0   2  11]
 [  0   4  37]
 [  0  10 141]]

Running Decision Tree model...
Best Params: {'min_samples_split': 20, 'min_samples_leaf': 4, 'max_depth': 20}
Decision Tree Model Metrics:
Train Accuracy:

...

Best Params: {'n_estimators': 150, 'learning_rate': 0.1}
AdaBoost Model Metrics:
Train Accuracy: 0.6933
Train Precision: 0.6149
Train Recall: 0.6933
Train F1 Score: 0.6296
Train Confusion Matrix:
[[  0  16  16]
 [  0  32 100]
 [  0  14 298]]

Test Accuracy: 0.7122
Test Precision: 0.6303
Test Recall: 0.7122
Test F1 Score: 0.6650
Test Confusion Matrix:
[[  0   6   7]
 [  0   8  33]
 [  0  13 138]]

Running Logistic Regression model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 10}
Logistic Regression Model Metrics:
Train Accuracy: 0.8298
Train Precision: 0.8309
Train Recall: 0.8298
Train F1 Score: 0.8200
Train Confusion Matrix:
[[ 17   6   9]
 [  1  80  51]
 [  0  14 298]]

Test Accuracy: 0.7512
Test Precision: 0.7282
Test Recall: 0.7512
Test F1 Score: 0.7369
Test Confusion Matrix:
[[  2   7   4]
 [  2  17  22]
 [  2  14 135]]

Running SVM model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'linear', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.8466
Train Precision: 0.8476
Train Recall: 0.8466
Train F1 Score: 0.8400
Train Confusion Matrix:
[[ 23   3   6]
 [  0  83  49]
 [  1  14 297]]

Test Accuracy: 0.7317
Test Precision: 0.7317
Test Recall: 0.7317
Test F1 Score: 0.7317
Test Confusion Matrix:
[[  3   6   4]
 [  3  19  19]
 [  7  16 128]]



In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

# Mount Google Drive
drive.mount('/content/drive')

# Load RoBERTa tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('roberta-base')
model = AutoModel.from_pretrained('roberta-base')

# Generate RoBERTa sentence embeddings
def generate_roberta_embeddings(sentences):
    sentence_vectors = []
    for sentence in sentences:
        # Convert sentence to string if it's not already
        if not isinstance(sentence, str):
            sentence = str(sentence)
        inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
            # Use the [CLS] token's embedding
            cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
        sentence_vectors.append(cls_embedding)
    return np.array(sentence_vectors)

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Individual model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    # Expanded parameter grid
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40, 50],
        'min_samples_split': [2, 5, 10, 15, 20],
        'min_samples_leaf': [1, 2, 4, 6, 8]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def catboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'iterations': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.3], 'depth': [4, 6, 10]}
    model = tune_model(CatBoostClassifier(verbose=0), param_grid, X_train, y_train)
    print("CatBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }
    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input file path
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_PREPROCESSED.xlsx'

# Load the dataset
df = pd.read_excel(input_file)

# Generate RoBERTa embeddings for the "answer" column
X = generate_roberta_embeddings(df['cleaned_answer'])

# Modify y to make sure the labels start from 0
y = df['confidence'].apply(lambda x: x + 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Run all models and print metrics
run_all_models(X_train, y_train, X_test, y_test)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Running KNN model...
Best Params: {'weights': 'uniform', 'n_neighbors': 1}
KNN Model Metrics:
Train Accuracy: 0.9748
Train Precision: 0.9750
Train Recall: 0.9748
Train F1 Score: 0.9749
Train Confusion Matrix:
[[ 31   0   1]
 [  0 128   4]
 [  0   7 305]]

Test Accuracy: 0.6683
Test Precision: 0.6877
Test Recall: 0.6683
Test F1 Score: 0.6756
Test Confusion Matrix:
[[  2   6   5]
 [  3  18  20]
 [  4  30 117]]

Running Random Forest model...
Best Params: {'n_estimators': 150, 'min_samples_split': 5, 'max_depth': 20}
Random Forest Model Metrics:
Train Accuracy: 0.9769
Train Precision: 0.9773
Train Recall: 0.9769
Train F1 Score: 0.9767
Train Confusion Matrix:
[[ 31   0   1]
 [  0 123   9]
 [  0   1 311]]

Test Accuracy: 0.7512
Test Precision: 0.6821
Test Recall: 0.7512
Test F1 Score: 0.6716
Test Confusion Matrix:
[[  0   1  12]
 [  0   5  36]
 [  0   2 149]]

Running Decision Tree model...
Best Params: {'min_samples_split': 15, 'min_samples_leaf': 1, 'max_depth': 30}
Decision Tree Model Me

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'adam', 'hidden_layer_sizes': (100,), 'activation': 'relu'}
MLP Model Metrics:
Train Accuracy: 0.8319
Train Precision: 0.8456
Train Recall: 0.8319
Train F1 Score: 0.8179
Train Confusion Matrix:
[[ 23   1   8]
 [  0  68  64]
 [  0   7 305]]

Test Accuracy: 0.7415
Test Precision: 0.6842
Test Recall: 0.7415
Test F1 Score: 0.7103
Test Confusion Matrix:
[[  0   4   9]
 [  1  16  24]
 [  0  15 136]]

Running XGBoost model...
Best Params: {'subsample': 0.5, 'n_estimators': 150, 'learning_rate': 0.2}
XGBoost Model Metrics:
Train Accuracy: 0.9769
Train Precision: 0.9773
Train Recall: 0.9769
Train F1 Score: 0.9767
Train Confusion Matrix:
[[ 31   0   1]
 [  0 123   9]
 [  0   1 311]]

Test Accuracy: 0.7561
Test Precision: 0.6773
Test Recall: 0.7561
Test F1 Score: 0.7013
Test Confusion Matrix:
[[  0   4   9]
 [  0  10  31]
 [  0   6 145]]

Running AdaBoost model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 9 is smaller than n_iter=30. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'n_estimators': 100, 'learning_rate': 1.0}
AdaBoost Model Metrics:
Train Accuracy: 0.8067
Train Precision: 0.8001
Train Recall: 0.8067
Train F1 Score: 0.8006
Train Confusion Matrix:
[[ 31   0   1]
 [  0  74  58]
 [  0  33 279]]

Test Accuracy: 0.6439
Test Precision: 0.6172
Test Recall: 0.6439
Test F1 Score: 0.6301
Test Confusion Matrix:
[[  0   4   9]
 [  0  13  28]
 [  2  30 119]]

Running Logistic Regression model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 10}
Logistic Regression Model Metrics:
Train Accuracy: 0.7962
Train Precision: 0.8116
Train Recall: 0.7962
Train F1 Score: 0.7734
Train Confusion Matrix:
[[ 13   5  14]
 [  0  61  71]
 [  0   7 305]]

Test Accuracy: 0.7366
Test Precision: 0.6609
Test Recall: 0.7366
Test F1 Score: 0.6898
Test Confusion Matrix:
[[  0   3  10]
 [  0  11  30]
 [  0  11 140]]

Running SVM model...
Best Params: {'kernel': 'linear', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.8277
Train Precision: 0.8488
Train Recall: 0.8277
Train F1 Score: 0.8107
Train Confusion Matrix:
[[ 22   1   9]
 [  0  64  68]
 [  0   4 308]]

Test Accuracy: 0.7512
Test Precision: 0.7428
Test Recall: 0.7512
Test F1 Score: 0.7136
Test Confusion Matrix:
[[  1   2  10]
 [  0  13  28]
 [  0  11 140]]



In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.decomposition import PCA

# Mount Google Drive
drive.mount('/content/drive')

# Load RoBERTa tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('roberta-base')
model = AutoModel.from_pretrained('roberta-base')

# Generate RoBERTa sentence embeddings
def generate_roberta_embeddings(sentences):
    sentence_vectors = []
    for sentence in sentences:
        # Ensure the sentence is a string
        sentence = str(sentence)  # Convert to string if not already
        inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
            # Use the [CLS] token's embedding
            cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
        sentence_vectors.append(cls_embedding)
    return np.array(sentence_vectors)

def apply_pca(X, variance_threshold=0.95):
    pca = PCA()
    X_reduced = pca.fit_transform(X)

    # Determine number of components to preserve desired variance
    explained_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(explained_variance >= variance_threshold) + 1
    print(f"Number of components selected: {n_components}")

    # Reapply PCA with the determined number of components
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)
    return X_reduced, pca

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Individual model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    # Expanded parameter grid
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40, 50],
        'min_samples_split': [2, 5, 10, 15, 20],
        'min_samples_leaf': [1, 2, 4, 6, 8]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def catboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'iterations': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.3], 'depth': [4, 6, 10]}
    model = tune_model(CatBoostClassifier(verbose=0), param_grid, X_train, y_train)
    print("CatBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }
    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input file path
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_PREPROCESSED.xlsx'

# Load the dataset
df = pd.read_excel(input_file)

# Generate RoBERTa embeddings for the "answer" column
X = generate_roberta_embeddings(df['cleaned_answer'])

# Apply PCA for dimensionality reduction
X_pca, pca_model = apply_pca(X, variance_threshold=0.95)

# Modify y to make sure the labels start from 0
y = df['confidence'].apply(lambda x: x + 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)

# Run all models and print metrics
run_all_models(X_train, y_train, X_test, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Number of components selected: 162
Running KNN model...
Best Params: {'weights': 'uniform', 'n_neighbors': 1}
KNN Model Metrics:
Train Accuracy: 0.9853
Train Precision: 0.9855
Train Recall: 0.9853
Train F1 Score: 0.9853
Train Confusion Matrix:
[[ 32   0   0]
 [  0 130   2]
 [  0   5 307]]

Test Accuracy: 0.6683
Test Precision: 0.6915
Test Recall: 0.6683
Test F1 Score: 0.6779
Test Confusion Matrix:
[[  3   6   4]
 [  5  16  20]
 [  2  31 118]]

Running Random Forest model...
Best Params: {'n_estimators': 50, 'min_samples_split': 5, 'max_depth': None}
Random Forest Model Metrics:
Train Accuracy: 0.9769
Train Precision: 0.9773
Train Recall: 0.9769
Train F1 Score: 0.9767
Train Confusion Matrix:
[[ 31   0   1]
 [  0 123   9]
 [  0   1 311]]

Test Accuracy: 0.7317
Test Precision: 0.6376
Test Recall: 0.7317
Test F1 Score: 0.6518
Test Confusion Matrix:
[[  0   0  13]
 [  0   4  37]
 [  0   5 146]]

Running Decision Tree model...
Best Params: {'min_samples_split': 20, 'min_samples_leaf': 4, 'ma

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'adam', 'hidden_layer_sizes': (100,), 'activation': 'relu'}
MLP Model Metrics:
Train Accuracy: 0.9748
Train Precision: 0.9754
Train Recall: 0.9748
Train F1 Score: 0.9746
Train Confusion Matrix:
[[ 30   0   2]
 [  0 123   9]
 [  0   1 311]]

Test Accuracy: 0.6683
Test Precision: 0.6417
Test Recall: 0.6683
Test F1 Score: 0.6544
Test Confusion Matrix:
[[  0   8   5]
 [  0  14  27]
 [  1  27 123]]

Running XGBoost model...
Best Params: {'subsample': 0.5, 'n_estimators': 100, 'learning_rate': 0.2}
XGBoost Model Metrics:
Train Accuracy: 0.9916
Train Precision: 0.9916
Train Recall: 0.9916
Train F1 Score: 0.9916
Train Confusion Matrix:
[[ 32   0   0]
 [  0 129   3]
 [  0   1 311]]

Test Accuracy: 0.7024
Test Precision: 0.6752
Test Recall: 0.7024
Test F1 Score: 0.6570
Test Confusion Matrix:
[[  1   5   7]
 [  0   6  35]
 [  0  14 137]]

Running AdaBoost model...


...

Best Params: {'n_estimators': 100, 'learning_rate': 1.0}
AdaBoost Model Metrics:
Train Accuracy: 0.7521
Train Precision: 0.7456
Train Recall: 0.7521
Train F1 Score: 0.7475
Train Confusion Matrix:
[[ 29   0   3]
 [  0  66  66]
 [  0  49 263]]

Test Accuracy: 0.6146
Test Precision: 0.5877
Test Recall: 0.6146
Test F1 Score: 0.6008
Test Confusion Matrix:
[[  0   7   6]
 [  0   8  33]
 [  1  32 118]]

Running Logistic Regression model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 10}
Logistic Regression Model Metrics:
Train Accuracy: 0.7626
Train Precision: 0.7777
Train Recall: 0.7626
Train F1 Score: 0.7290
Train Confusion Matrix:
[[ 12   5  15]
 [  0  47  85]
 [  0   8 304]]

Test Accuracy: 0.7317
Test Precision: 0.6454
Test Recall: 0.7317
Test F1 Score: 0.6750
Test Confusion Matrix:
[[  0   3  10]
 [  0   8  33]
 [  0   9 142]]

Running SVM model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'rbf', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.9748
Train Precision: 0.9754
Train Recall: 0.9748
Train F1 Score: 0.9746
Train Confusion Matrix:
[[ 30   0   2]
 [  0 123   9]
 [  0   1 311]]

Test Accuracy: 0.6976
Test Precision: 0.6566
Test Recall: 0.6976
Test F1 Score: 0.6761
Test Confusion Matrix:
[[  0   4   9]
 [  2  14  25]
 [  2  20 129]]



In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

# Mount Google Drive
drive.mount('/content/drive')

# Load RoBERTa tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('roberta-base')
model = AutoModel.from_pretrained('roberta-base')

# Generate RoBERTa sentence embeddings
def generate_roberta_embeddings(sentences):
    sentence_vectors = []
    for sentence in sentences:
        # Convert sentence to string if it's not already
        if not isinstance(sentence, str):
            sentence = str(sentence)
        inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
            # Use the [CLS] token's embedding
            cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
        sentence_vectors.append(cls_embedding)
    return np.array(sentence_vectors)

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Individual model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    param_grid = {'max_depth': [None, 10, 20, 30, 40, 50], 'min_samples_split': [2, 5, 10, 15, 20], 'min_samples_leaf': [1, 2, 4, 6, 8]}
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }
    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input file path
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_PREPROCESSED.xlsx'

# Load the dataset
df = pd.read_excel(input_file)

# Select 3 additional features
additional_features = ['filler_ratio', 'avg_word_length', 'sentence_length_variation']  # Replace 'other_feature' with the third feature's name

# Normalize the additional features
normalized_features = (df[additional_features] - df[additional_features].mean()) / df[additional_features].std()

# Generate RoBERTa embeddings for the "answer" column
roberta_embeddings = generate_roberta_embeddings(df['cleaned_answer'])

# Concatenate RoBERTa embeddings with the selected features
X = np.hstack((roberta_embeddings, normalized_features.values))

# Modify y to make sure the labels start from 0
y = df['confidence'].apply(lambda x: x + 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Run all models and print metrics
run_all_models(X_train, y_train, X_test, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Running KNN model...
Best Params: {'weights': 'distance', 'n_neighbors': 20}
KNN Model Metrics:
Train Accuracy: 0.9916
Train Precision: 0.9916
Train Recall: 0.9916
Train F1 Score: 0.9916
Train Confusion Matrix:
[[ 32   0   0]
 [  0 129   3]
 [  0   1 311]]

Test Accuracy: 0.7268
Test Precision: 0.5996
Test Recall: 0.7268
Test F1 Score: 0.6386
Test Confusion Matrix:
[[  0   2  11]
 [  0   2  39]
 [  0   4 147]]

Running Random Forest model...
Best Params: {'n_estimators': 50, 'min_samples_split': 5, 'max_depth': 30}
Random Forest Model Metrics:
Train Accuracy: 0.9832
Train Precision: 0.9833
Train Recall: 0.9832
Train F1 Score: 0.9831
Train Confusion Matrix:
[[ 31   0   1]
 [  0 127   5]
 [  0   2 310]]

Test Accuracy: 0.7366
Test Precision: 0.6516
Test Recall: 0.7366
Test F1 Score: 0.6721
Test Confusion Matrix:
[[  0   1  12]
 [  0   7  34]
 [  0   7 144]]

Running Decision Tree model...
Best Params: {'min_samples_split': 20, 'min_samples_leaf': 4, 'max_depth': 20}
Decision Tree Model M

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'adam', 'hidden_layer_sizes': (50,), 'activation': 'tanh'}
MLP Model Metrics:
Train Accuracy: 0.9055
Train Precision: 0.9071
Train Recall: 0.9055
Train F1 Score: 0.9031
Train Confusion Matrix:
[[ 27   1   4]
 [  0 101  31]
 [  0   9 303]]

Test Accuracy: 0.7171
Test Precision: 0.6737
Test Recall: 0.7171
Test F1 Score: 0.6946
Test Confusion Matrix:
[[  0   6   7]
 [  1  16  24]
 [  1  19 131]]

Running XGBoost model...
Best Params: {'subsample': 0.5, 'n_estimators': 50, 'learning_rate': 0.2}
XGBoost Model Metrics:
Train Accuracy: 0.9895
Train Precision: 0.9897
Train Recall: 0.9895
Train F1 Score: 0.9894
Train Confusion Matrix:
[[ 32   0   0]
 [  0 127   5]
 [  0   0 312]]

Test Accuracy: 0.7610
Test Precision: 0.6854
Test Recall: 0.7610
Test F1 Score: 0.7001
Test Confusion Matrix:
[[  0   3  10]
 [  0   9  32]
 [  0   4 147]]

Running Logistic Regression model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'liblinear', 'penalty': 'l2', 'C': 10}
Logistic Regression Model Metrics:
Train Accuracy: 0.7626
Train Precision: 0.7758
Train Recall: 0.7626
Train F1 Score: 0.7299
Train Confusion Matrix:
[[ 10   6  16]
 [  0  50  82]
 [  0   9 303]]

Test Accuracy: 0.7463
Test Precision: 0.6677
Test Recall: 0.7463
Test F1 Score: 0.6934
Test Confusion Matrix:
[[  0   3  10]
 [  0  10  31]
 [  0   8 143]]

Running SVM model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'linear', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.8340
Train Precision: 0.8478
Train Recall: 0.8340
Train F1 Score: 0.8208
Train Confusion Matrix:
[[ 22   1   9]
 [  0  70  62]
 [  0   7 305]]

Test Accuracy: 0.7317
Test Precision: 0.6600
Test Recall: 0.7317
Test F1 Score: 0.6894
Test Confusion Matrix:
[[  0   3  10]
 [  0  12  29]
 [  0  13 138]]



In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

# Mount Google Drive
drive.mount('/content/drive')

# Load RoBERTa tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('roberta-base')
model = AutoModel.from_pretrained('roberta-base')

# Generate RoBERTa sentence embeddings
def generate_roberta_embeddings(sentences):
    sentence_vectors = []
    for sentence in sentences:
        # Convert sentence to string if it's not already
        if not isinstance(sentence, str):
            sentence = str(sentence)
        inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
            # Use the [CLS] token's embedding
            cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
        sentence_vectors.append(cls_embedding)
    return np.array(sentence_vectors)

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Individual model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    param_grid = {'max_depth': [None, 10, 20, 30, 40, 50], 'min_samples_split': [2, 5, 10, 15, 20], 'min_samples_leaf': [1, 2, 4, 6, 8]}
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }
    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input file path
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_PREPROCESSED.xlsx'

# Load the dataset
df = pd.read_excel(input_file)

# Select 3 additional features
additional_features = ['filler_ratio', 'avg_word_length', 'sentence_length_variation']  # Replace 'other_feature' with the third feature's name

# Normalize the additional features
normalized_features = (df[additional_features] - df[additional_features].mean()) / df[additional_features].std()

# Generate RoBERTa embeddings for the "answer" column
roberta_embeddings = generate_roberta_embeddings(df['answer'])

# Concatenate RoBERTa embeddings with the selected features
X = np.hstack((roberta_embeddings, normalized_features.values))

# Modify y to make sure the labels start from 0
y = df['confidence'].apply(lambda x: x + 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Run all models and print metrics
run_all_models(X_train, y_train, X_test, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Running KNN model...
Best Params: {'weights': 'uniform', 'n_neighbors': 5}
KNN Model Metrics:
Train Accuracy: 0.7416
Train Precision: 0.7248
Train Recall: 0.7416
Train F1 Score: 0.7264
Train Confusion Matrix:
[[ 10   8  14]
 [  5  64  63]
 [  5  28 279]]

Test Accuracy: 0.6780
Test Precision: 0.6451
Test Recall: 0.6780
Test F1 Score: 0.6600
Test Confusion Matrix:
[[  0   4   9]
 [  4  11  26]
 [  5  18 128]]

Running Random Forest model...
Best Params: {'n_estimators': 50, 'min_samples_split': 10, 'max_depth': 20}
Random Forest Model Metrics:
Train Accuracy: 0.9937
Train Precision: 0.9938
Train Recall: 0.9937
Train F1 Score: 0.9937
Train Confusion Matrix:
[[ 32   0   0]
 [  0 129   3]
 [  0   0 312]]

Test Accuracy: 0.7512
Test Precision: 0.6717
Test Recall: 0.7512
Test F1 Score: 0.6978
Test Confusion Matrix:
[[  0   4   9]
 [  0  10  31]
 [  0   7 144]]

Running Decision Tree model...
Best Params: {'min_samples_split': 2, 'min_samples_leaf': 4, 'max_depth': 40}
Decision Tree Model Met

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'adam', 'hidden_layer_sizes': (50,), 'activation': 'relu'}
MLP Model Metrics:
Train Accuracy: 0.9139
Train Precision: 0.9154
Train Recall: 0.9139
Train F1 Score: 0.9117
Train Confusion Matrix:
[[ 27   2   3]
 [  0 103  29]
 [  1   6 305]]

Test Accuracy: 0.7512
Test Precision: 0.7341
Test Recall: 0.7512
Test F1 Score: 0.7412
Test Confusion Matrix:
[[  4   4   5]
 [  4  16  21]
 [  3  14 134]]

Running XGBoost model...
Best Params: {'subsample': 0.5, 'n_estimators': 150, 'learning_rate': 0.2}
XGBoost Model Metrics:
Train Accuracy: 0.9958
Train Precision: 0.9958
Train Recall: 0.9958
Train F1 Score: 0.9958
Train Confusion Matrix:
[[ 32   0   0]
 [  0 130   2]
 [  0   0 312]]

Test Accuracy: 0.7366
Test Precision: 0.7027
Test Recall: 0.7366
Test F1 Score: 0.7103
Test Confusion Matrix:
[[  2   6   5]
 [  1  12  28]
 [  1  13 137]]

Running Logistic Regression model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 10}
Logistic Regression Model Metrics:
Train Accuracy: 0.8929
Train Precision: 0.8943
Train Recall: 0.8929
Train F1 Score: 0.8887
Train Confusion Matrix:
[[ 21   6   5]
 [  0  99  33]
 [  1   6 305]]

Test Accuracy: 0.7659
Test Precision: 0.7465
Test Recall: 0.7659
Test F1 Score: 0.7537
Test Confusion Matrix:
[[  2   6   5]
 [  2  20  19]
 [  2  14 135]]

Running SVM model...
Best Params: {'kernel': 'linear', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.9097
Train Precision: 0.9113
Train Recall: 0.9097
Train F1 Score: 0.9071
Train Confusion Matrix:
[[ 29   1   2]
 [  0 100  32]
 [  1   7 304]]

Test Accuracy: 0.7366
Test Precision: 0.7243
Test Recall: 0.7366
Test F1 Score: 0.7301
Test Confusion Matrix:
[[  2   8   3]
 [  3  17  21]
 [  6  13 132]]



In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.decomposition import PCA

# Mount Google Drive
drive.mount('/content/drive')

# Load RoBERTa tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('roberta-base')
model = AutoModel.from_pretrained('roberta-base')

# Generate RoBERTa sentence embeddings
def generate_roberta_embeddings(sentences):
    sentence_vectors = []
    for sentence in sentences:
        # Convert sentence to string if it's not already
        if not isinstance(sentence, str):
            sentence = str(sentence)
        inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
            # Use the [CLS] token's embedding
            cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
        sentence_vectors.append(cls_embedding)
    return np.array(sentence_vectors)

def apply_pca(X, variance_threshold=0.95):
    pca = PCA()
    X_reduced = pca.fit_transform(X)

    # Determine number of components to preserve desired variance
    explained_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(explained_variance >= variance_threshold) + 1
    print(f"Number of components selected: {n_components}")

    # Reapply PCA with the determined number of components
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)
    return X_reduced, pca

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Individual model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    param_grid = {'max_depth': [None, 10, 20, 30, 40, 50], 'min_samples_split': [2, 5, 10, 15, 20], 'min_samples_leaf': [1, 2, 4, 6, 8]}
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }
    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input file path
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_PREPROCESSED.xlsx'

# Load the dataset
df = pd.read_excel(input_file)

# Select 3 additional features
additional_features = ['filler_ratio', 'avg_word_length', 'sentence_length_variation']  # Replace 'other_feature' with the third feature's name

# Normalize the additional features
normalized_features = (df[additional_features] - df[additional_features].mean()) / df[additional_features].std()

# Generate RoBERTa embeddings for the "answer" column
roberta_embeddings = generate_roberta_embeddings(df['answer'])

# Concatenate RoBERTa embeddings with the selected features
X = np.hstack((roberta_embeddings, normalized_features.values))

# Apply PCA for dimensionality reduction
X_pca, pca_model = apply_pca(X, variance_threshold=0.95)

# Modify y to make sure the labels start from 0
y = df['confidence'].apply(lambda x: x + 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)

# Run all models and print metrics
run_all_models(X_train, y_train, X_test, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Number of components selected: 19
Running KNN model...
Best Params: {'weights': 'uniform', 'n_neighbors': 7}
KNN Model Metrics:
Train Accuracy: 0.7248
Train Precision: 0.7015
Train Recall: 0.7248
Train F1 Score: 0.7033
Train Confusion Matrix:
[[  5   8  19]
 [  4  63  65]
 [  2  33 277]]

Test Accuracy: 0.6732
Test Precision: 0.6427
Test Recall: 0.6732
Test F1 Score: 0.6576
Test Confusion Matrix:
[[  0   5   8]
 [  0  14  27]
 [  5  22 124]]

Running Random Forest model...
Best Params: {'n_estimators': 50, 'min_samples_split': 2, 'max_depth': None}
Random Forest Model Metrics:
Train Accuracy: 0.9958
Train Precision: 0.9958
Train Recall: 0.9958
Train F1 Score: 0.9958
Train Confusion Matrix:
[[ 32   0   0]
 [  0 130   2]
 [  0   0 312]]

Test Accuracy: 0.7317
Test Precision: 0.6603
Test Recall: 0.7317
Test F1 Score: 0.6900
Test Confusion Matrix:
[[  0   6   7]
 [  0  11  30]
 [  1  11 139]]

Running Decision Tree model...
Best Params: {'min_samples_split': 5, 'min_samples_leaf': 6, 'max_

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'adam', 'hidden_layer_sizes': (100,), 'activation': 'tanh'}
MLP Model Metrics:
Train Accuracy: 0.7416
Train Precision: 0.7216
Train Recall: 0.7416
Train F1 Score: 0.7218
Train Confusion Matrix:
[[  8  14  10]
 [  4  61  67]
 [  2  26 284]]

Test Accuracy: 0.7073
Test Precision: 0.6927
Test Recall: 0.7073
Test F1 Score: 0.6994
Test Confusion Matrix:
[[  3   6   4]
 [  4  13  24]
 [  3  19 129]]

Running XGBoost model...
Best Params: {'subsample': 1.0, 'n_estimators': 150, 'learning_rate': 0.01}
XGBoost Model Metrics:
Train Accuracy: 0.9664
Train Precision: 0.9680
Train Recall: 0.9664
Train F1 Score: 0.9659
Train Confusion Matrix:
[[ 30   0   2]
 [  0 118  14]
 [  0   0 312]]

Test Accuracy: 0.7317
Test Precision: 0.6799
Test Recall: 0.7317
Test F1 Score: 0.7000
Test Confusion Matrix:
[[  0   5   8]
 [  3  12  26]
 [  3  10 138]]

Running Logistic Regression model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 10}
Logistic Regression Model Metrics:
Train Accuracy: 0.7101
Train Precision: 0.6840
Train Recall: 0.7101
Train F1 Score: 0.6748
Train Confusion Matrix:
[[  7  15  10]
 [  2  42  88]
 [  1  22 289]]

Test Accuracy: 0.7317
Test Precision: 0.7072
Test Recall: 0.7317
Test F1 Score: 0.7168
Test Confusion Matrix:
[[  3   5   5]
 [  2  14  25]
 [  3  15 133]]

Running SVM model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'rbf', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.8361
Train Precision: 0.8459
Train Recall: 0.8361
Train F1 Score: 0.8216
Train Confusion Matrix:
[[ 16   6  10]
 [  1  74  57]
 [  0   4 308]]

Test Accuracy: 0.7073
Test Precision: 0.6633
Test Recall: 0.7073
Test F1 Score: 0.6830
Test Confusion Matrix:
[[  0   6   7]
 [  5  11  25]
 [  2  15 134]]



In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.decomposition import PCA

# Mount Google Drive
drive.mount('/content/drive')

# Load RoBERTa tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('roberta-base')
model = AutoModel.from_pretrained('roberta-base')

# Generate RoBERTa sentence embeddings
def generate_roberta_embeddings(sentences):
    sentence_vectors = []
    for sentence in sentences:
        # Convert sentence to string if it's not already
        if not isinstance(sentence, str):
            sentence = str(sentence)
        inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
            # Use the [CLS] token's embedding
            cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
        sentence_vectors.append(cls_embedding)
    return np.array(sentence_vectors)

def apply_pca(X, variance_threshold=0.95):
    pca = PCA()
    X_reduced = pca.fit_transform(X)

    # Determine number of components to preserve desired variance
    explained_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(explained_variance >= variance_threshold) + 1
    print(f"Number of components selected: {n_components}")

    # Reapply PCA with the determined number of components
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)
    return X_reduced, pca

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Individual model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    param_grid = {'max_depth': [None, 10, 20, 30, 40, 50], 'min_samples_split': [2, 5, 10, 15, 20], 'min_samples_leaf': [1, 2, 4, 6, 8]}
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }
    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input file path
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_PREPROCESSED.xlsx'

# Load the dataset
df = pd.read_excel(input_file)

# Select 3 additional features
additional_features = ['filler_ratio', 'avg_word_length', 'sentence_length_variation']  # Replace 'other_feature' with the third feature's name

# Normalize the additional features
normalized_features = (df[additional_features] - df[additional_features].mean()) / df[additional_features].std()

# Generate RoBERTa embeddings for the "answer" column
roberta_embeddings = generate_roberta_embeddings(df['cleaned_answer'])

# Concatenate RoBERTa embeddings with the selected features
X = np.hstack((roberta_embeddings, normalized_features.values))

# Apply PCA for dimensionality reduction
X_pca, pca_model = apply_pca(X, variance_threshold=0.95)

# Modify y to make sure the labels start from 0
y = df['confidence'].apply(lambda x: x + 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)

# Run all models and print metrics
run_all_models(X_train, y_train, X_test, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Number of components selected: 8
Running KNN model...
Best Params: {'weights': 'distance', 'n_neighbors': 15}
KNN Model Metrics:
Train Accuracy: 0.9916
Train Precision: 0.9916
Train Recall: 0.9916
Train F1 Score: 0.9916
Train Confusion Matrix:
[[ 32   0   0]
 [  0 130   2]
 [  0   2 310]]

Test Accuracy: 0.7317
Test Precision: 0.6306
Test Recall: 0.7317
Test F1 Score: 0.6610
Test Confusion Matrix:
[[  0   3  10]
 [  0   5  36]
 [  0   6 145]]

Running Random Forest model...
Best Params: {'n_estimators': 100, 'min_samples_split': 5, 'max_depth': 20}
Random Forest Model Metrics:
Train Accuracy: 0.9916
Train Precision: 0.9917
Train Recall: 0.9916
Train F1 Score: 0.9916
Train Confusion Matrix:
[[ 32   0   0]
 [  0 128   4]
 [  0   0 312]]

Test Accuracy: 0.7220
Test Precision: 0.6260
Test Recall: 0.7220
Test F1 Score: 0.6596
Test Confusion Matrix:
[[  0   3  10]
 [  0   6  35]
 [  0   9 142]]

Running Decision Tree model...
Best Params: {'min_samples_split': 20, 'min_samples_leaf': 6, 'max

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'adam', 'hidden_layer_sizes': (50,), 'activation': 'tanh'}
MLP Model Metrics:
Train Accuracy: 0.6912
Train Precision: 0.6617
Train Recall: 0.6912
Train F1 Score: 0.6388
Train Confusion Matrix:
[[  5   7  20]
 [  2  31  99]
 [  1  18 293]]

Test Accuracy: 0.7220
Test Precision: 0.6364
Test Recall: 0.7220
Test F1 Score: 0.6599
Test Confusion Matrix:
[[  0   1  12]
 [  1   6  34]
 [  1   8 142]]

Running XGBoost model...
Best Params: {'subsample': 0.5, 'n_estimators': 150, 'learning_rate': 0.01}
XGBoost Model Metrics:
Train Accuracy: 0.8508
Train Precision: 0.8709
Train Recall: 0.8508
Train F1 Score: 0.8362
Train Confusion Matrix:
[[ 13   4  15]
 [  0  80  52]
 [  0   0 312]]

Test Accuracy: 0.7317
Test Precision: 0.6442
Test Recall: 0.7317
Test F1 Score: 0.6686
Test Confusion Matrix:
[[  0   3  10]
 [  1   6  34]
 [  1   6 144]]

Running Logistic Regression model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 10}
Logistic Regression Model Metrics:
Train Accuracy: 0.6681
Train Precision: 0.6203
Train Recall: 0.6681
Train F1 Score: 0.6021
Train Confusion Matrix:
[[  2   6  24]
 [  2  24 106]
 [  0  20 292]]

Test Accuracy: 0.7366
Test Precision: 0.6583
Test Recall: 0.7366
Test F1 Score: 0.6730
Test Confusion Matrix:
[[  0   1  12]
 [  0   7  34]
 [  1   6 144]]

Running SVM model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'rbf', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.7080
Train Precision: 0.7348
Train Recall: 0.7080
Train F1 Score: 0.6411
Train Confusion Matrix:
[[  1   3  28]
 [  0  32 100]
 [  0   8 304]]

Test Accuracy: 0.7171
Test Precision: 0.6041
Test Recall: 0.7171
Test F1 Score: 0.6398
Test Confusion Matrix:
[[  0   1  12]
 [  1   3  37]
 [  0   7 144]]



In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.model_selection import RandomizedSearchCV
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.decomposition import PCA

# Mount Google Drive
drive.mount('/content/drive')

# Load RoBERTa tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('roberta-base')
model = AutoModel.from_pretrained('roberta-base')

# Generate RoBERTa sentence embeddings
def generate_roberta_embeddings(sentences):
    sentence_vectors = []
    for sentence in sentences:
        inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
            cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
        sentence_vectors.append(cls_embedding)
    return np.array(sentence_vectors)

# PCA for dimensionality reduction
def apply_pca(X, variance_threshold=0.95):
    pca = PCA()
    X_reduced = pca.fit_transform(X)
    explained_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(explained_variance >= variance_threshold) + 1
    print(f"Number of components selected: {n_components}")
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)
    return X_reduced, pca

# Print performance metrics
def print_metrics(y_true, y_pred, dataset_type="Test"):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    rec = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
    conf_matrix = confusion_matrix(y_true, y_pred)
    print(f"{dataset_type} Accuracy: {acc:.4f}")
    print(f"{dataset_type} Precision: {prec:.4f}")
    print(f"{dataset_type} Recall: {rec:.4f}")
    print(f"{dataset_type} F1 Score: {f1:.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{conf_matrix}\n")
    return acc

# Performance table function
def performance_table(y_train, train_pred, y_test, test_pred, model_name):
    train_acc = accuracy_score(y_train, train_pred)
    test_acc = accuracy_score(y_test, test_pred)
    train_test_gap = abs(train_acc - test_acc)
    # Get unique labels from both y_test and test_pred
    all_labels = sorted(list(set(y_test) | set(test_pred)))
    # Ensure both one-hot encoded arrays have the same columns
    y_test_dummies = pd.get_dummies(y_test, columns=all_labels).reindex(columns=all_labels, fill_value=0)
    test_pred_dummies = pd.get_dummies(test_pred, columns=all_labels).reindex(columns=all_labels, fill_value=0)
    roc_auc = roc_auc_score(y_test_dummies, test_pred_dummies, multi_class="ovr", average="weighted")

    overfitting = "Yes" if train_test_gap > 0.1 else "No"
    print(f"\nPerformance Table for {model_name}:")
    print(f"{'Metric':<20}{'Value':<10}")
    print(f"{'-'*30}")
    print(f"{'Train Accuracy':<20}{train_acc:.2%}")
    print(f"{'Test Accuracy':<20}{test_acc:.2%}")
    print(f"{'Train-Test Gap':<20}{train_test_gap:.2%}")
    print(f"{'ROC-AUC':<20}{roc_auc:.2f}")
    print(f"{'Overfitting':<20}{overfitting}")
    print("\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Individual model functions
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    print_metrics(y_train, train_pred, "Train")
    print_metrics(y_test, test_pred, "Test")
    performance_table(y_train, train_pred, y_test, test_pred, "KNN")

def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    print_metrics(y_train, train_pred, "Train")
    print_metrics(y_test, test_pred, "Test")
    performance_table(y_train, train_pred, y_test, test_pred, "Random Forest")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    print_metrics(y_train, train_pred, "Train")
    print_metrics(y_test, test_pred, "Test")
    performance_table(y_train, train_pred, y_test, test_pred, "Logistic Regression")

def decision_tree_model(X_train, y_train, X_test, y_test):
    param_grid = {'max_depth': [None, 10, 20, 30, 40, 50], 'min_samples_split': [2, 5, 10, 15, 20], 'min_samples_leaf': [1, 2, 4, 6, 8]}
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    # Calculate and assign values to train_pred and test_pred
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    print_metrics(y_train, train_pred, "Train")
    print_metrics(y_test, test_pred, "Test")
    performance_table(y_train, train_pred, y_test, test_pred, "Decision Tree")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    print_metrics(y_train, train_pred, "Train")
    print_metrics(y_test, test_pred, "Test")
    performance_table(y_train, train_pred, y_test, test_pred,"Naive Bayes")


def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    print_metrics(y_train, train_pred, "Train")
    print_metrics(y_test, test_pred, "Test")
    performance_table(y_train, train_pred, y_test, test_pred, "MLP model")


def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    print_metrics(y_train, train_pred, "Train")
    print_metrics(y_test, test_pred, "Test")
    performance_table(y_train, train_pred, y_test, test_pred,"XG Boost")


def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    print_metrics(y_train, train_pred, "Train")
    print_metrics(y_test, test_pred, "Test")
    performance_table(y_train, train_pred, y_test, test_pred,"SVM")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    print_metrics(y_train, train_pred, "Train")
    print_metrics(y_test, test_pred, "Test")
    performance_table(y_train, train_pred, y_test, test_pred,"SVM")


# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }
    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input file path
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_CONFIDENCE.xlsx'

# Load the dataset
df = pd.read_excel(input_file)

# Generate RoBERTa embeddings for the "answer" column
X = generate_roberta_embeddings(df['answer'])

# Apply PCA for dimensionality reduction
X_pca, pca_model = apply_pca(X, variance_threshold=0.95)

# Modify y to make sure the labels start from 0
y = df['confidence'].apply(lambda x: x + 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)

# Run all models and print metrics
run_all_models(X_train, y_train, X_test, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Number of components selected: 135
Running KNN model...
Best Params: {'weights': 'uniform', 'n_neighbors': 3}
Train Accuracy: 0.8256
Train Precision: 0.8292
Train Recall: 0.8256
Train F1 Score: 0.8187
Train Confusion Matrix:
[[ 24   4   4]
 [ 16  74  42]
 [  4  13 295]]

Test Accuracy: 0.6829
Test Precision: 0.6813
Test Recall: 0.6829
Test F1 Score: 0.6801
Test Confusion Matrix:
[[  3   7   3]
 [  9   9  23]
 [  9  14 128]]


Performance Table for KNN:
Metric              Value     
------------------------------
Train Accuracy      82.56%
Test Accuracy       68.29%
Train-Test Gap      14.27%
ROC-AUC             0.65
Overfitting         Yes


Running Random Forest model...
Best Params: {'n_estimators': 50, 'min_samples_split': 2, 'max_depth': 30}
Train Accuracy: 0.9958
Train Precision: 0.9958
Train Recall: 0.9958
Train F1 Score: 0.9958
Train Confusion Matrix:
[[ 32   0   0]
 [  0 130   2]
 [  0   0 312]]

Test Accuracy: 0.7463
Test Precision: 0.7210
Test Recall: 0.7463
Test F1 Score: 0

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'adam', 'hidden_layer_sizes': (100,), 'activation': 'relu'}
MLP Model Metrics:
Train Accuracy: 0.9916
Train Precision: 0.9917
Train Recall: 0.9916
Train F1 Score: 0.9916
Train Confusion Matrix:
[[ 32   0   0]
 [  0 128   4]
 [  0   0 312]]

Test Accuracy: 0.7268
Test Precision: 0.7225
Test Recall: 0.7268
Test F1 Score: 0.7246
Test Confusion Matrix:
[[  2   9   2]
 [  4  17  20]
 [  5  16 130]]


Performance Table for MLP model:
Metric              Value     
------------------------------
Train Accuracy      99.16%
Test Accuracy       72.68%
Train-Test Gap      26.48%
ROC-AUC             0.70
Overfitting         Yes


Running XGBoost model...
Best Params: {'subsample': 0.5, 'n_estimators': 50, 'learning_rate': 0.2}
XGBoost Model Metrics:
Train Accuracy: 0.9958
Train Precision: 0.9958
Train Recall: 0.9958
Train F1 Score: 0.9958
Train Confusion Matrix:
[[ 32   0   0]
 [  0 130   2]
 [  0   0 312]]

Test Accuracy: 0.7512
Test Precision: 0.7073
Test Recall: 0.7512
T

...

Best Params: {'n_estimators': 150, 'learning_rate': 0.1}
AdaBoost Model Metrics:
Train Accuracy: 0.6954
Train Precision: 0.6165
Train Recall: 0.6954
Train F1 Score: 0.6314
Train Confusion Matrix:
[[  0  17  15]
 [  0  32 100]
 [  0  13 299]]

Test Accuracy: 0.7171
Test Precision: 0.6377
Test Recall: 0.7171
Test F1 Score: 0.6709
Test Confusion Matrix:
[[  0   5   8]
 [  0   9  32]
 [  0  13 138]]


Performance Table for SVM:
Metric              Value     
------------------------------
Train Accuracy      69.54%
Test Accuracy       71.71%
Train-Test Gap      2.17%
ROC-AUC             0.57
Overfitting         No


Running Logistic Regression model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 10}
Train Accuracy: 0.8340
Train Precision: 0.8358
Train Recall: 0.8340
Train F1 Score: 0.8233
Train Confusion Matrix:
[[ 17   6   9]
 [  1  79  52]
 [  1  10 301]]

Test Accuracy: 0.7463
Test Precision: 0.7245
Test Recall: 0.7463
Test F1 Score: 0.7335
Test Confusion Matrix:
[[  2   7   4]
 [  2  17  22]
 [  3  14 134]]


Performance Table for Logistic Regression:
Metric              Value     
------------------------------
Train Accuracy      83.40%
Test Accuracy       74.63%
Train-Test Gap      8.77%
ROC-AUC             0.68
Overfitting         No


Running SVM model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'rbf', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.9895
Train Precision: 0.9897
Train Recall: 0.9895
Train F1 Score: 0.9894
Train Confusion Matrix:
[[ 32   0   0]
 [  0 127   5]
 [  0   0 312]]

Test Accuracy: 0.7415
Test Precision: 0.7230
Test Recall: 0.7415
Test F1 Score: 0.7314
Test Confusion Matrix:
[[  2   7   4]
 [  5  16  20]
 [  2  15 134]]


Performance Table for SVM:
Metric              Value     
------------------------------
Train Accuracy      98.95%
Test Accuracy       74.15%
Train-Test Gap      24.80%
ROC-AUC             0.69
Overfitting         Yes


